In [1]:
import pandas as pd

# load data
data = pd.read_json('../data/preprocessed/combined_data.json')
# where label is train_tr
data = data[data['label'] == 'train_tr'].reset_index(drop=True)

In [2]:
data.shape

(5269, 6)

In [3]:
# METHODOLOGY: Make triplets by db_id where all negative examples are in the same db_id with 10 random negative examples

# List to hold JSON strings
jsonl_output = []

# Grouping the dataframe by db_id
for db_id, group in data.groupby('db_id'):
    # Iterate over each row in the group
    for i, row in group.iterrows():
        # Get negative evidence, excluding the current row
        neg_evidence = group[group.index != i]['evidence']
        # If there are fewer than 10 negatives, take as many as are available; otherwise, sample 10
        if len(neg_evidence) <= 10:
            sampled_neg = neg_evidence.tolist()
        else:
            sampled_neg = neg_evidence.sample(10).tolist()  # Each sample is independent

        # Constructing the JSON object for each row
        json_object = {
            'set': {
                'query': row['question'],
                'pos': [row['evidence']],
                'neg': sampled_neg
            }
        }
        jsonl_output.append(json_object)

In [4]:
# save to jsonl with indent
import json
with open('../data/preprocessed/training_evidence_triplet_data.json', 'w') as f:
    for json_object in jsonl_output:
        json.dump(json_object, f)
        f.write('\n')


In [5]:
# METHODOLOGY: Make triplets by db_id where all negative examples are in the same db_id with 10 random negative examples

# List to hold JSON strings
jsonl_output = []

# Grouping the dataframe by db_id
for db_id, group in data.groupby('db_id'):
    # Iterate over each row in the group
    for i, row in group.iterrows():
        # Get negative evidence, excluding the current row
        neg_evidence = group[group.index != i]['SQL']
        # If there are fewer than 10 negatives, take as many as are available; otherwise, sample 10
        if len(neg_evidence) <= 10:
            sampled_neg = neg_evidence.tolist()
        else:
            sampled_neg = neg_evidence.sample(10).tolist()  # Each sample is independent

        # Constructing the JSON object for each row
        json_object = {
            'set': {
                'query': row['question'],
                'pos': [row['SQL']],
                'neg': sampled_neg
            }
        }
        jsonl_output.append(json_object)

In [6]:
# save to jsonl with indent
import json
with open('../data/preprocessed/training_sql_triplet_data.json', 'w') as f:
    for json_object in jsonl_output:
        json.dump(json_object, f)
        f.write('\n')
